In [1]:
import tensorflow as tf; print(tf.__version__)

2.3.4


In [2]:
import nest_asyncio
nest_asyncio.apply()
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from tqdm import tqdm
from pathlib import Path

from matplotlib import pyplot as plt

In [3]:
def plot_graph(X, y, format = '-', label=''):
    plt.plot(X, y, format, label=label)
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.grid(True)
np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

experiment_name = "xray_image_dataset"
method = "tff_training"
this_dir = Path.cwd()
model_dir = this_dir / "saved_models" / experiment_name / method
output_dir = this_dir / "results" / experiment_name / method

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data generators
train_dir='./../Data/archive/Data/train/'
validation_dir='./../Data/archive/Data/test/'
batch_size=10

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen =  ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=batch_size,
        # Since we use categorical_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


In [6]:
train_generator.class_indices.values()

dict_values([0, 1, 2])

In [7]:
# Store the data in X_train, y_train variables by iterating over the batches
train_generator.reset()
X_train, y_train = next(train_generator)
for i in tqdm(range(int(len(train_generator)/batch_size)-1)): #1st batch is already fetched before the for loop.
    img, label = next(train_generator)
    X_train = np.append(X_train, img, axis=0 )
    y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:08<00:00,  5.94it/s]

(510, 150, 150, 3) (510, 3)


In [8]:
validation_generator.reset()
X_test, y_test = next(validation_generator)
for i in tqdm(range(int(len(validation_generator)/batch_size)-1)): #1st batch is already fetched before the for loop.
    img, label = next(validation_generator)
    X_test = np.append(X_test, img, axis=0 )
    y_test = np.append(y_test, label, axis=0)
    
print(X_test.shape, y_test.shape)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  7.71it/s]

(120, 150, 150, 3) (120, 3)


In [9]:
X_test = X_test.astype(np.float32).reshape(360, 150, 150, 1)
y_test = y_test.astype(np.int32).reshape(360, 1)



In [10]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.int32)

In [11]:

split=4 #Number of clients 
total_image_count = len(X_train)
image_per_set = int(np.floor(total_image_count/split))

In [12]:
client_train_dataset = collections.OrderedDict()
for i in range(1, split+1):
    client_name = "client_" + str(i)
    start = image_per_set * (i-1)
    end = image_per_set * i

    print(f"Adding data from {start} to {end} for client : {client_name}")
    data = collections.OrderedDict((('label', y_train[start:end]), ('pixels', X_train[start:end])))
    client_train_dataset[client_name] = data



Adding data from 0 to 127 for client : client_1
Adding data from 127 to 254 for client : client_2
Adding data from 254 to 381 for client : client_3
Adding data from 381 to 508 for client : client_4


In [13]:

from tensorflow import reshape, nest, config
NUM_EPOCHS=5
BATCH_SIZE=10
PREFETCH_BUFFER = 10

print(len(client_train_dataset))


train_dataset = tff.simulation.FromTensorSlicesClientData(client_train_dataset)

sample_dataset = train_dataset.create_tf_dataset_for_client(train_dataset.client_ids[0])
#print(sample_dataset['pixels'])
sample_element = next(iter(sample_dataset))

SHUFFLE_BUFFER = image_per_set

def preprocess(dataset):

    def batch_format_fn(element):
        """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
        print(element['pixels'].get_shape())
        return collections.OrderedDict(
            x=reshape(element['pixels'], [-1, 150, 150, 1]),
            y=reshape(element['label'], [-1, 1]))
    
    
    
    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
        BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        

preprocessed_sample_dataset = preprocess(sample_dataset)
sample_batch = nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_sample_dataset)))


def make_federated_data(client_data, client_ids):
    return [preprocess(client_data.create_tf_dataset_for_client(x)) for x in client_ids]

federated_train_data = make_federated_data(train_dataset, train_dataset.client_ids)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))



4
(None, 150, 150, 3)
(None, 150, 150, 3)
(None, 150, 150, 3)
(None, 150, 150, 3)
(None, 150, 150, 3)
Number of client datasets: 4
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 150, 150, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>


In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense


def create_keras_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),
        MaxPool2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

In [15]:

import collections
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow import reshape, nest, config
from tensorflow.keras import losses, metrics, optimizers
import tensorflow_federated as tff
from matplotlib import pyplot as plt
from pathlib import Path

method = "tff_training"
client_lr = 1e-2
server_lr = 1e-2
NUM_ROUNDS = 10


def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.

    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_sample_dataset.element_spec,
      loss=losses.SparseCategoricalCrossentropy(),
      metrics=[metrics.SparseCategoricalAccuracy()])


iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: optimizers.Adam(learning_rate=client_lr),
    server_optimizer_fn=lambda: optimizers.SGD(learning_rate=server_lr))

print(str(iterative_process.initialize.type_signature))

state = iterative_process.initialize()



( -> <model=<trainable=<float32[3,3,1,32],float32[32],float32[3,3,32,64],float32[64],float32[82944,512],float32[512],float32[512,10],float32[10]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)


In [16]:
tff_train_loss.append(float(tff_metrics.loss))
tff_val_loss.append(ev_result[0])

metric_collection = {"sparse_categorical_accuracy": tff_train_acc,
                     "val_sparse_categorical_accuracy": tff_val_acc,
                     "loss": tff_train_loss,
                     "val_loss": tff_val_loss}

#if eval_model:
#    eval_model.save(model_dir / (experiment_name + ".h5"))
#else:
#    print("training didn't started")
#    exit()

fig = plt.figure(figsize=(10, 6))
plot_graph(list(range(1, 51))[4::5], tff_train_acc, label='Train Accuracy')
plot_graph(list(range(1, 51))[4::5], tff_val_acc, label='Validation Accuracy')
plt.legend()
plt.savefig(output_dir / "federated_model_Accuracy.png")

plt.figure(figsize=(10, 6))
plot_graph(list(range(1, 51))[4::5], tff_train_loss, label='Train loss')
plot_graph(list(range(1, 51))[4::5], tff_val_loss, label='Validation loss')
plt.legend()
plt.savefig(output_dir / "federated_model_loss.png")



# saving metric values to text file

txt_file_path = output_dir / (experiment_name + ".txt")
with open(txt_file_path.as_posix(), "w") as handle:
    content = []
    for key, val in metric_collection.items():
        line_content = key
        val = [str(k) for k in val]
        line_content = line_content + " " + " ".join(val)
        content.append(line_content)
    handle.write("\n".join(content))


NameError: name 'tff_train_loss' is not defined

In [23]:
# @Author: sreedevic
# @Date:   2021-11-08T10:20:45+00:00
# @Last modified by:   sreedevic
# @Last modified time: 2021-11-08T22:44:17+00:00



#!/usr/bin/env python
# coding: utf-8

# In[ ]:



import nest_asyncio
nest_asyncio.apply()
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from tqdm import tqdm
from pathlib import Path

from matplotlib import pyplot as plt


def plot_graph(X, y, format = '-', label=''):
    plt.plot(X, y, format, label=label)
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.grid(True)
np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

experiment_name = "xray_image_dataset"
method = "tff_training"
this_dir = Path.cwd()
model_dir = this_dir / "saved_models" / experiment_name / method
output_dir = this_dir / "results" / experiment_name / method

# In[69]:


from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data generators
train_dir='./../Data/archive/Data/train/'
validation_dir='./../Data/archive/Data/test/'
batch_size=10

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen =  ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 32x32
        target_size=(32, 32),
        batch_size=batch_size,
        # Since we use categorical_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical')


# In[43]:


train_generator.class_indices.values()


# In[44]:


import tensorflow as tf; print(tf.__version__)


# In[45]:


# Store the data in X_train, y_train variables by iterating over the batches
train_generator.reset()
X_train, y_train = next(train_generator)
for i in tqdm(range(int(len(train_generator)/batch_size)-1)): #1st batch is already fetched before the for loop.
    img, label = next(train_generator)
    X_train = np.append(X_train, img, axis=0 )
    y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)


# In[62]:


validation_generator.reset()
X_test, y_test = next(validation_generator)
for i in tqdm(range(int(len(validation_generator)/batch_size)-1)): #1st batch is already fetched before the for loop.
    img, label = next(validation_generator)
    X_test = np.append(X_test, img, axis=0 )
    y_test = np.append(y_test, label, axis=0)

print(X_test.shape, y_test.shape)


# In[67]:

X_test = X_test.astype(np.float32).reshape(360, 32, 32, 1)
y_test = y_test.astype(np.int32).reshape(360, 1)


# In[46]:


len(train_generator)


# In[47]:


X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.int32)


# In[48]:


split=4 #Number of clients
total_image_count = len(X_train)
image_per_set = int(np.floor(total_image_count/split))


# In[49]:


client_train_dataset = collections.OrderedDict()
for i in range(1, split+1):
    client_name = "client_" + str(i)
    start = image_per_set * (i-1)
    end = image_per_set * i

    print(f"Adding data from {start} to {end} for client : {client_name}")
    data = collections.OrderedDict((('label', y_train[start:end]), ('pixels', X_train[start:end])))
    client_train_dataset[client_name] = data


# In[55]:


from tensorflow import reshape, nest, config
NUM_EPOCHS=5
BATCH_SIZE=10
PREFETCH_BUFFER = 10

print(len(client_train_dataset))


train_dataset = tff.simulation.FromTensorSlicesClientData(client_train_dataset)

sample_dataset = train_dataset.create_tf_dataset_for_client(train_dataset.client_ids[0])
#print(sample_dataset['pixels'])
sample_element = next(iter(sample_dataset))

SHUFFLE_BUFFER = image_per_set

def preprocess(dataset):

    def batch_format_fn(element):
        """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
        print(element['pixels'].get_shape())
        return collections.OrderedDict(
            x=reshape(element['pixels'], [-1, 32, 32, 1]),
            y=reshape(element['label'], [-1, 1]))



    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
        BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)


preprocessed_sample_dataset = preprocess(sample_dataset)
sample_batch = nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_sample_dataset)))


def make_federated_data(client_data, client_ids):
    return [preprocess(client_data.create_tf_dataset_for_client(x)) for x in client_ids]

federated_train_data = make_federated_data(train_dataset, train_dataset.client_ids)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))


# In[56]:


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout


def create_keras_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
        #MaxPool2D(pool_size=(2, 2)),
        #Conv2D(32, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(3, activation='softmax')

    ])
    return model


# In[61]:


import collections
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow import reshape, nest, config
from tensorflow.keras import losses, metrics, optimizers
import tensorflow_federated as tff
from matplotlib import pyplot as plt
from pathlib import Path

method = "tff_training"
client_lr = 1e-2
server_lr = 1e-2
NUM_ROUNDS = 10


def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.

    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_sample_dataset.element_spec,
      loss=losses.SparseCategoricalCrossentropy(),
      metrics=[metrics.SparseCategoricalAccuracy()])


iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: optimizers.Adam(learning_rate=client_lr),
    server_optimizer_fn=lambda: optimizers.SGD(learning_rate=server_lr))

print(str(iterative_process.initialize.type_signature))

state = iterative_process.initialize()

tff_train_acc = []
tff_val_acc = []
tff_train_loss = []
tff_val_loss = []

eval_model = None
for round_num in range(1, NUM_ROUNDS+1):
    state, tff_metrics = iterative_process.next(state, federated_train_data)
    eval_model = create_keras_model()
    eval_model.compile(optimizer=optimizers.Adam(learning_rate=client_lr),
                       loss=losses.SparseCategoricalCrossentropy(),
                       metrics=[metrics.SparseCategoricalAccuracy()])


    tff.learning.assign_weights_to_keras_model(eval_model, state.model)
#     tff.learning.set_weights(eval_model, state.model)

    ev_result = eval_model.evaluate(X_test, y_test, verbose=0)
    print('round {:2d}, metrics={}'.format(round_num, tff_metrics))
    print(f"Eval loss : {ev_result[0]} and Eval accuracy : {ev_result[1]}")
    tff_train_acc.append(float(tff_metrics.sparse_categorical_accuracy))
    tff_val_acc.append(ev_result[1])
    tff_train_loss.append(float(tff_metrics.loss))
    tff_val_loss.append(ev_result[0])

metric_collection = {"sparse_categorical_accuracy": tff_train_acc,
                     "val_sparse_categorical_accuracy": tff_val_acc,
                     "loss": tff_train_loss,
                     "val_loss": tff_val_loss}

#if eval_model:
#    eval_model.save(model_dir / (experiment_name + ".h5"))
#else:
#    print("training didn't started")
#    exit()

fig = plt.figure(figsize=(10, 6))
plot_graph(list(range(1, 51))[4::5], tff_train_acc, label='Train Accuracy')
plot_graph(list(range(1, 51))[4::5], tff_val_acc, label='Validation Accuracy')
plt.legend()
plt.savefig(output_dir / "federated_model_Accuracy.png")

plt.figure(figsize=(10, 6))
plot_graph(list(range(1, 51))[4::5], tff_train_loss, label='Train loss')
plot_graph(list(range(1, 51))[4::5], tff_val_loss, label='Validation loss')
plt.legend()
plt.savefig(output_dir / "federated_model_loss.png")



# saving metric values to text file

txt_file_path = output_dir / (experiment_name + ".txt")
with open(txt_file_path.as_posix(), "w") as handle:
    content = []
    for key, val in metric_collection.items():
        line_content = key
        val = [str(k) for k in val]
        line_content = line_content + " " + " ".join(val)
        content.append(line_content)
    handle.write("\n".join(content))


# In[ ]:


Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.
2.3.4


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 10.25it/s]


(510, 32, 32, 3) (510, 3)


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 13.92it/s]


(120, 32, 32, 3) (120, 3)
Adding data from 0 to 127 for client : client_1
Adding data from 127 to 254 for client : client_2
Adding data from 254 to 381 for client : client_3
Adding data from 381 to 508 for client : client_4
4
(None, 32, 32, 3)
(None, 32, 32, 3)
(None, 32, 32, 3)
(None, 32, 32, 3)
(None, 32, 32, 3)
Number of client datasets: 4
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 32, 32, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>
( -> <model=<trainable=<float32[3,3,1,32],float32[32],float32[7200,512],float32[512],float32[512,3],float32[3]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)


AttributeError: module 'tensorflow_federated.python.learning' has no attribute 'set_weights'